In [1]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()
GECKO_API = os.getenv("GECKO_API")

url = "https://api.coingecko.com/api/v3/derivatives/exchanges"

headers = {
    "accept": "application/json",
    "x-cg-api-key": GECKO_API
}

response = requests.get(url, headers=headers)
data = response.json()

# Extract just the IDs
exchange_ids = [ex["id"] for ex in data]

print("Available Exchange IDs:")
for ex_id in exchange_ids:
    print("-", ex_id)


Available Exchange IDs:
- binance_futures
- bitget_futures
- bybit
- gate_futures
- coinw_futures
- hyperliquid
- weex-futures
- xt_derivatives
- okex_swap
- huobi_dm
- coincatch_derivatives
- toobit_derivatives
- bitvenus
- bitmart_futures
- mxc_futures
- whitebit_futures
- bingx_futures
- deepcoin_derivatives
- kumex
- lbank-futures


In [9]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()
GECKO_API = os.getenv("GECKO_API")

BASE_URL = "https://api.coingecko.com/api/v3/derivatives/exchanges"

headers = {
    "accept": "application/json",
    "x-cg-api-key": GECKO_API
}

# Step 1: Get all derivative exchanges
response = requests.get(BASE_URL, headers=headers)
exchanges = response.json()

all_symbols = set()

# Step 2: Loop through each exchange and fetch tickers
for ex in exchanges:
    ex_id = ex["id"]
    url = f"{BASE_URL}/{ex_id}?include_tickers=all"
    
    r = requests.get(url, headers=headers)
    if r.status_code != 200:
        print(f"⚠️ Failed to fetch {ex_id}")
        continue
    
    data = r.json()
    tickers = data.get("tickers", [])
    
    # Collect base tokens (or use "symbol" if you want pair names)
    for t in tickers:
        if t.get("contract_type") == "perpetual":
            all_symbols.add(t.get("base"))

    # Stop if we already collected 50 unique symbols
    if len(all_symbols) >= 50:
        break

# Step 3: Print first 100 unique token symbols
print("✅ First 100 Available Token Symbols:")
for s in sorted(list(all_symbols))[:100]:
    print("-", s)


✅ First 100 Available Token Symbols:
- 1000000BOB
- 1000000MOG
- 1000BONK
- 1000CAT
- 1000CHEEMS
- 1000FLOKI
- 1000LUNC
- 1000PEPE
- 1000RATS
- 1000SATS
- 1000SHIB
- 1000WHY
- 1000X
- 1000XEC
- 1INCH
- 1MBABYDOGE
- A
- A2Z
- AAVE
- ACE
- ACH
- ACT
- ACX
- ADA
- AERGO
- AERO
- AEVO
- AGLD
- AGT
- AI
- AI16Z
- AIN
- AIO
- AIOT
- AIXBT
- AKT
- ALCH
- ALGO
- ALICE
- ALPHA
- ALPINE
- ALT
- ANIME
- ANKR
- APE
- API3
- APT
- AR
- ARB
- ARC
- ARK
- ARKM
- ARPA
- ASR
- ASTR
- ATA
- ATH
- ATOM
- AUCTION
- AVA
- AVAAI
- AVAX
- AWE
- AXL
- AXS
- B
- B2
- B3
- BABY
- BAKE
- BAN
- BANANA
- BANANAS31
- BAND
- BANK
- BAT
- BB
- BCH
- BDXN
- BEAMX
- BEL
- BERA
- BICO
- BID
- BIGTIME
- BIO
- BLUR
- BMT
- BNB
- BNT
- BOME
- BR
- BRETT
- BROCCOLI714
- BROCCOLIF3B
- BSV
- BSW
- BTC
- BTCDOM
- BULLA


In [7]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()
GECKO_API = os.getenv("GECKO_API")

BASE_URL = "https://api.coingecko.com/api/v3/derivatives/exchanges"

headers = {
    "accept": "application/json",
    "x-cg-api-key": GECKO_API
}

# Step 1: Get all derivative exchanges
response = requests.get(BASE_URL, headers=headers)
exchanges = response.json()

all_symbols = set()

# Step 2: Loop through each exchange and fetch tickers
for ex in exchanges:
    ex_id = ex["id"]
    url = f"{BASE_URL}/{ex_id}?include_tickers=all"
    
    r = requests.get(url, headers=headers)
    if r.status_code != 200:
        print(f"⚠️ Failed to fetch {ex_id}")
        continue
    
    data = r.json()
    tickers = data.get("tickers", [])
    
    # Collect base tokens (or symbol if you want pair names like BTCUSDT)
    for t in tickers:
        if t.get("contract_type") == "perpetual":
            all_symbols.add(t.get("base"))   # use "symbol" instead if you want full pair

# Step 3: Print unique token symbols
print("✅ Available Token Symbols:")
for s in sorted(all_symbols):
    print("-", s)


ConnectionError: HTTPSConnectionPool(host='api.coingecko.com', port=443): Max retries exceeded with url: /api/v3/derivatives/exchanges (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002A977AE0B90>: Failed to resolve 'api.coingecko.com' ([Errno 11001] getaddrinfo failed)"))

In [2]:
import requests

url = "https://api.coingecko.com/api/v3/derivatives/exchanges/binance_futures?include_tickers=all"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.json())

{'name': 'Binance (Futures)', 'open_interest_btc': 376210.48, 'trade_volume_24h_btc': '1471071.83', 'number_of_perpetual_pairs': 549, 'number_of_futures_pairs': 31, 'image': 'https://coin-images.coingecko.com/markets/images/466/small/binance_futures.jpg?1706864452', 'year_established': 2019, 'country': None, 'description': '', 'url': 'https://www.binance.com/', 'tickers': [{'symbol': '1000000BOBUSDT', 'base': '1000000BOB', 'target': 'USDT', 'trade_url': 'https://www.binance.com/en/futures/1000000BOBUSDT', 'contract_type': 'perpetual', 'last': 0.069, 'h24_percentage_change': 6.908, 'index': 0.06891899, 'index_basis_percentage': 0.071, 'bid_ask_spread': 0.0002901494269549706, 'funding_rate': 0.01, 'open_interest_usd': 1779748.7813628598, 'h24_volume': 55645560.0, 'converted_volume': {'btc': '33.10213263635779434740365868504249359020764334546976447344', 'eth': '808.5493501824676472192478304644606817565094285803404331715497786350708944', 'usd': '3830157.77461444704951115416'}, 'converted_l

In [ ]:
import requests
import pandas as pd

# List of exchange IDs you want to compare
exchange_ids = ["binance_futures", "bybit", "bitget_futures", "coinw_futures",
                "gate_futures", "hyperliquid", "weex-futures", "okex_swap",
                "huobi_dm", "mxc_futures", "bingx_futures", "lbank-futures"]

BASE_URL = "https://api.coingecko.com/api/v3/derivatives/exchanges"

all_data = []

for ex_id in exchange_ids:
    url = f"{BASE_URL}/{ex_id}?include_tickers=all"
mxc_futures
bingx_futures
lbank-futures]

BASE_URL = "https://api.coingecko.com/api/v3/derivatives/exchanges"

all_data = []

for ex_id in exchange_ids:
    url = f"{BASE_URL}/{ex_id}?include_tickers=all"
    headers = {"accept": "application/json"}
    r = requests.get(url, headers=headers)

    if r.status_code != 200:
        print(f"⚠️ Failed to fetch {ex_id}")
        continue

    data = r.json()
    tickers = data.get("tickers", [])

    for t in tickers:
        if t.get("contract_type") == "perpetual":  # Only perpetual contracts
            all_data.append({
                "exchange_id": ex_id,
                "exchange_name": data.get("name"),
                "symbol": t.get("symbol"),
                "base": t.get("base"),
                "target": t.get("target"),
                "funding_rate": t.get("funding_rate"),
                "open_interest_usd": t.get("open_interest_usd"),
                "h24_volume": t.get("h24_volume"),
                "last": t.get("last"),
                "trade_url": t.get("trade_url")
            })

# Convert into DataFrame
df = pd.DataFrame(all_data)

# Show top rows
print(df.head(10000))

# Save to CSV for later use
df.to_csv("perps_funding_comparison.csv", index=False)
print("✅ Saved to perps_funding_comparison.csv")


          exchange_id      exchange_name          symbol        base target  \
0     binance_futures  Binance (Futures)  1000000BOBUSDT  1000000BOB   USDT   
1     binance_futures  Binance (Futures)  1000000MOGUSDT  1000000MOG   USDT   
2     binance_futures  Binance (Futures)    1000BONKUSDC    1000BONK   USDC   
3     binance_futures  Binance (Futures)    1000BONKUSDT    1000BONK   USDT   
4     binance_futures  Binance (Futures)     1000CATUSDT     1000CAT   USDT   
...               ...                ...             ...         ...    ...   
1987    coinw_futures    CoinW (Futures)        XRP/USDT         XRP   USDT   
1988    coinw_futures    CoinW (Futures)        YGG/USDT         YGG   USDT   
1989    coinw_futures    CoinW (Futures)       ZETA/USDT        ZETA   USDT   
1990    coinw_futures    CoinW (Futures)         ZK/USDT          ZK   USDT   
1991    coinw_futures    CoinW (Futures)        ZRO/USDT         ZRO   USDT   

      funding_rate  open_interest_usd    h24_volume

In [4]:
print("API Key:", GECKO_API)


API Key: CG-tEqpaCHxWu8hm5kFHc7AbC98


In [10]:
import requests
import pandas as pd

# ✅ Allowed symbols
allowed_symbols = {
    "BTC", "ETH", "XRP", "USDT", "BNB", "SOL", "USDC", "DOGE", "TRX", "ADA", "LINK",
    "WBETH", "HYPE", "SUI", "XLM", "BCH", "AVAX", "HBAR", "LTC", "LEO", "TON", "SHIB",
    "UNI", "WBT", "DOT", "BGB", "AAVE", "CRO", "ENA", "XMR", "PEPE", "OKB", "MNT",
    "DAI", "ETC", "NEAR", "APT", "ARB", "PI", "ICP", "POL", "ATOM", "PENGU", "SEI",
    "BONK", "TRUMP", "JUP", "OP", "PUMP", "RAY", "CAKE", "FLOKI"
}

# ✅ List of exchanges
exchange_ids = [
    "binance_futures", "bybit", "bitget_futures", "coinw_futures",
    "gate_futures", "hyperliquid", "weex-futures", "okex_swap",
    "huobi_dm", "mxc_futures", "bingx_futures", "lbank-futures"
]

BASE_URL = "https://api.coingecko.com/api/v3/derivatives/exchanges"
all_data = []

# ✅ Fetch data from exchanges
for ex_id in exchange_ids:
    url = f"{BASE_URL}/{ex_id}?include_tickers=all"
    headers = {"accept": "application/json"}
    r = requests.get(url, headers=headers)

    if r.status_code != 200:
        print(f"⚠️ Failed to fetch {ex_id}")
        continue

    data = r.json()
    tickers = data.get("tickers", [])

    for t in tickers:
        if t.get("contract_type") == "perpetual":
            base_symbol = t.get("base", "").upper()
            if base_symbol in allowed_symbols:  # ✅ filter only allowed tokens
                all_data.append({
                    "exchange_id": ex_id,
                    "exchange_name": data.get("name"),
                    "symbol": t.get("symbol"),
                    "base": base_symbol,
                    "target": t.get("target"),
                    "funding_rate": t.get("funding_rate"),
                    "open_interest_usd": t.get("open_interest_usd"),
                    "h24_volume": t.get("h24_volume"),
                    "last": t.get("last"),
                    "trade_url": t.get("trade_url")
                })

# ✅ Convert into DataFrame
df = pd.DataFrame(all_data)

# ✅ Save all results in one CSV (your database)
filename = "funding_comparison_database.csv"
df.to_csv(filename, index=False)

print(f"✅ Saved {len(df)} rows to {filename}")


⚠️ Failed to fetch hyperliquid
⚠️ Failed to fetch weex-futures
⚠️ Failed to fetch okex_swap
⚠️ Failed to fetch huobi_dm
⚠️ Failed to fetch mxc_futures
⚠️ Failed to fetch bingx_futures
⚠️ Failed to fetch lbank-futures
✅ Saved 348 rows to funding_comparison_database.csv


In [12]:

import pandas as pd

csv_path = r"C:\Users\FEMI\Desktop\Funding & Perps Dashboard\Notebook\funding_comparison_database.csv"
df = pd.read_csv(csv_path)

print(df.head(500))

         exchange_id      exchange_name        symbol  base target  \
0    binance_futures  Binance (Futures)  AAVEUSD_PERP  AAVE    USD   
1    binance_futures  Binance (Futures)      AAVEUSDC  AAVE   USDC   
2    binance_futures  Binance (Futures)      AAVEUSDT  AAVE   USDT   
3    binance_futures  Binance (Futures)   ADAUSD_PERP   ADA    USD   
4    binance_futures  Binance (Futures)       ADAUSDC   ADA   USDC   
..               ...                ...           ...   ...    ...   
343     gate_futures     Gate (Futures)      TRX_USDT   TRX   USDT   
344     gate_futures     Gate (Futures)      UNI_USDT   UNI   USDT   
345     gate_futures     Gate (Futures)     USDC_USDT  USDC   USDT   
346     gate_futures     Gate (Futures)      XLM_USDT   XLM   USDT   
347     gate_futures     Gate (Futures)      XRP_USDT   XRP   USDT   

     funding_rate  open_interest_usd    h24_volume     last  \
0           0.010       2.505351e+08  1.037233e+05  352.830   
1           0.010       6.226941e

In [13]:
import pandas as pd

# ✅ Path to your saved database CSV
csv_path = r"C:\Users\FEMI\Desktop\Funding & Perps Dashboard\Notebook\funding_comparison_database.csv"

# Load data
df = pd.read_csv(csv_path)

# Ask user for a token symbol
symbol_to_compare = input("Enter the symbol to compare (e.g., BTC, ETH, ADA): ").strip().upper()

# Filter for selected symbol
df_symbol = df[df["base"].str.upper() == symbol_to_compare]

# Select required columns
result = df_symbol[["exchange_id", "symbol", "funding_rate", "open_interest_usd"]]

# Print results
print("\n📊 Funding Comparison for:", symbol_to_compare)
print(result)

# Optionally save output
# output_file = f"{symbol_to_compare}_funding_comparison.csv"
# result.to_csv(output_file, index=False)
# print(f"✅ Saved filtered results to {output_file}")



📊 Funding Comparison for: ETH
         exchange_id               symbol  funding_rate  open_interest_usd
34   binance_futures          ETHUSD_PERP          0.01       1.378269e+09
35   binance_futures              ETHUSDC          0.01       1.150704e+09
36   binance_futures              ETHUSDT          0.01       1.096465e+10
112            bybit               ETHUSD          0.01       4.364366e+08
113            bybit              ETHPERP          0.01       7.513262e+07
114            bybit              ETHUSDT          0.01       5.350244e+09
205   bitget_futures        ETHPERP_CMCBL          0.01       0.000000e+00
206   bitget_futures         ETHUSD_DMCBL          0.01       1.359277e+09
207   bitget_futures  ETHUSD_DMCBL_250627          0.00       3.922523e+05
208   bitget_futures  ETHUSD_DMCBL_250926          0.00       1.844452e+07
209   bitget_futures  ETHUSD_DMCBL_251226          0.00       9.422267e+06
210   bitget_futures        ETHUSDT_UMCBL          0.01       5.59604

In [ ]:
import requests
import pandas as pd
import time
import os

# ✅ Save location for your database
csv_path = r"C:\Users\FEMI\Desktop\Funding & Perps Dashboard\Notebook\funding_comparison_database.csv"

# ✅ Exchanges to check
exchange_ids = [
    "binance_futures", "bybit", "bitget_futures", "coinw_futures",
    "gate_futures", "hyperliquid", "weex-futures", "okex_swap",
    "huobi_dm", "mxc_futures", "bingx_futures", "lbank-futures"
]

# ✅ Tokens you want to keep
allowed_tokens = {
    "BTC","ETH","XRP","USDT","BNB","SOL","USDC","DOGE","TRX","ADA",
    "LINK","WBETH","HYPE","SUI","XLM","BCH","AVAX","HBAR","LTC","LEO",
    "TON","SHIB","UNI","WBT","DOT","BGB","AAVE","CRO","ENA","XMR","PEPE",
    "OKB","MNT","DAI","ETC","NEAR","APT","ARB","PI","ICP","POL","ATOM",
    "PENGU","SEI","BONK","TRUMP","JUP","OP","PUMP","RAY","CAKE","FLOKI"
}

BASE_URL = "https://api.coingecko.com/api/v3/derivatives/exchanges"

def fetch_data():
    all_data = []
    for ex_id in exchange_ids:
        url = f"{BASE_URL}/{ex_id}?include_tickers=all"
        headers = {"accept": "application/json"}
        try:
            r = requests.get(url, headers=headers, timeout=30)
            r.raise_for_status()
        except Exception as e:
            print(f"⚠️ Failed to fetch {ex_id}: {e}")
            continue

        data = r.json()
        tickers = data.get("tickers", [])

        for t in tickers:
            if t.get("contract_type") == "perpetual" and t.get("base") in allowed_tokens:
                all_data.append({
                    "exchange_id": ex_id,
                    "exchange_name": data.get("name"),
                    "symbol": t.get("symbol"),
                    "base": t.get("base"),
                    "target": t.get("target"),
                    "funding_rate": t.get("funding_rate"),
                    "open_interest_usd": t.get("open_interest_usd"),
                    "h24_volume": t.get("h24_volume"),
                    "last": t.get("last"),
                    "trade_url": t.get("trade_url")
                })
    return pd.DataFrame(all_data)

def main():
    while True:
        print("\n🔄 Fetching new data...")
        df = fetch_data()

        if not df.empty:
            df.to_csv(csv_path, index=False)
            print(f"✅ Database updated and saved to {csv_path} at {time.strftime('%Y-%m-%d %H:%M:%S')}")
        else:
            print("⚠️ No data fetched, skipping save.")

        # Sleep for 4 hours (14400 seconds)
        print("⏳ Sleeping for 4 hours...")
        time.sleep(14400)

if __name__ == "__main__":
    main()


In [14]:
import requests
import pandas as pd

# ✅ Allowed symbols
allowed_symbols = {
    "BTC", "ETH", "XRP", "USDT", "BNB", "SOL", "USDC", "DOGE", "TRX", "ADA", "LINK",
    "WBETH", "HYPE", "SUI", "XLM", "BCH", "AVAX", "HBAR", "LTC", "LEO", "TON", "SHIB",
    "UNI", "WBT", "DOT", "BGB", "AAVE", "CRO", "ENA", "XMR", "PEPE", "OKB", "MNT",
    "DAI", "ETC", "NEAR", "APT", "ARB", "PI", "ICP", "POL", "ATOM", "PENGU", "SEI",
    "BONK", "TRUMP", "JUP", "OP", "PUMP", "RAY", "CAKE", "FLOKI"
}

# ✅ List of exchanges to fetch
exchange_ids = [
    "binance_futures", "bybit", "bitget_futures", "coinw_futures",
    "gate_futures", "hyperliquid", "weex-futures", "okex_swap",
    "huobi_dm", "mxc_futures", "bingx_futures", "lbank-futures"
]

BASE_URL = "https://api.coingecko.com/api/v3/derivatives/exchanges"
all_data = []

# ✅ Fetch data from each exchange
for ex_id in exchange_ids:
    url = f"{BASE_URL}/{ex_id}?include_tickers=all"
    headers = {"accept": "application/json"}
    r = requests.get(url, headers=headers)

    if r.status_code != 200:
        print(f"⚠️ Failed to fetch {ex_id}")
        continue

    data = r.json()
    tickers = data.get("tickers", [])

    for t in tickers:
        if t.get("contract_type") == "perpetual":
            base_symbol = t.get("base", "").upper()
            if base_symbol in allowed_symbols:  # ✅ only keep allowed tokens
                all_data.append({
                    "exchange_id": ex_id,
                    "exchange_name": data.get("name"),
                    "symbol": t.get("symbol"),
                    "base": base_symbol,
                    "target": t.get("target"),
                    "funding_rate": t.get("funding_rate"),
                    "open_interest_usd": t.get("open_interest_usd"),
                    "h24_volume": t.get("h24_volume"),
                    "last": t.get("last"),
                    "trade_url": t.get("trade_url")
                })

# ✅ Convert into DataFrame
df = pd.DataFrame(all_data)

# ✅ Ask user which token to compare
symbol_to_compare = input("Enter the symbol to compare (e.g., BTC, ETH, ADA): ").strip().upper()

if symbol_to_compare not in allowed_symbols:
    print(f"❌ {symbol_to_compare} is not in the allowed list.")
else:
    df_symbol = df[df["base"] == symbol_to_compare]

    # Show funding_rate and open_interest_usd across exchanges for this symbol
    result = df_symbol[["exchange_id", "symbol", "funding_rate", "open_interest_usd"]]

    print(result)

    # Save to CSV
    filename = f"{symbol_to_compare}_funding_comparison.csv"
    result.to_csv(filename, index=False)
    print(f"✅ Saved to {filename}")


SSLError: HTTPSConnectionPool(host='api.coingecko.com', port=443): Max retries exceeded with url: /api/v3/derivatives/exchanges/bybit?include_tickers=all (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_BAD_RECORD_MAC] sslv3 alert bad record mac (_ssl.c:2657)')))

In [4]:
import requests
import pandas as pd

# List of exchange IDs you want to compare
exchange_ids = ["binance_futures", "bybit", "bitget_futures", "coinw_futures"]

BASE_URL = "https://api.coingecko.com/api/v3/derivatives/exchanges"

all_data = []

for ex_id in exchange_ids:
    url = f"{BASE_URL}/{ex_id}?include_tickers=all"
    headers = {"accept": "application/json"}
    r = requests.get(url, headers=headers)

    if r.status_code != 200:
        print(f"⚠️ Failed to fetch {ex_id}")
        continue

    data = r.json()
    tickers = data.get("tickers", [])

    for t in tickers:
        if t.get("contract_type") == "perpetual":
            all_data.append({
                "exchange_id": ex_id,
                "exchange_name": data.get("name"),
                "symbol": t.get("symbol"),
                "base": t.get("base"),
                "target": t.get("target"),
                "funding_rate": t.get("funding_rate"),
                "open_interest_usd": t.get("open_interest_usd"),
                "h24_volume": t.get("h24_volume"),
                "last": t.get("last"),
                "trade_url": t.get("trade_url")
            })

# Convert into DataFrame
df = pd.DataFrame(all_data)

# Filter for a specific symbol (e.g., "bitcoin" as base)
# symbol_to_compare = "ADA"
# df_symbol = df[df["base"].str.lower() == symbol_to_compare.lower()]

# Filter for a specific symbol (e.g., "bitcoin" as base)
symbol_to_compare = input("Enter the symbol to compare (e.g., bitcoin, eth, ada): ").strip()
df_symbol = df[df["base"].str.lower() == symbol_to_compare.lower()]

# Show funding_rate and open_interest_usd across exchanges for this symbol
result = df_symbol[["exchange_id", "symbol", "funding_rate", "open_interest_usd"]]

print(result)

# Optionally, save the filtered result
result.to_csv(f"{symbol_to_compare}_funding_comparison.csv", index=False)
print(f"✅ Saved to {symbol_to_compare}_funding_comparison.csv")

          exchange_id         symbol  funding_rate  open_interest_usd
430   binance_futures    SOLUSD_PERP         0.010       1.959491e+08
431   binance_futures        SOLUSDC         0.010       1.126399e+08
432   binance_futures        SOLUSDT         0.010       2.291499e+09
1059            bybit         SOLUSD         0.010       5.638069e+07
1060            bybit        SOLPERP         0.010       1.533085e+07
1061            bybit        SOLUSDT         0.010       1.573076e+09
1723   bitget_futures  SOLPERP_CMCBL         0.010       0.000000e+00
1724   bitget_futures   SOLUSD_DMCBL         0.010       2.096455e+08
1725   bitget_futures  SOLUSDT_UMCBL         0.012       1.367450e+09
1968    coinw_futures       SOL/USDT         0.010       6.679367e+08
✅ Saved to SOL_funding_comparison.csv


In [6]:
import pandas as pd

csv_path = r"C:\Users\FEMI\Desktop\Funding & Perps Dashboard\Notebook\SOL_funding_comparison.csv"
df = pd.read_csv(csv_path)

print(df.head(20))

       exchange_id         symbol  funding_rate  open_interest_usd
0  binance_futures    SOLUSD_PERP         0.010       1.959491e+08
1  binance_futures        SOLUSDC         0.010       1.126399e+08
2  binance_futures        SOLUSDT         0.010       2.291499e+09
3            bybit         SOLUSD         0.010       5.638069e+07
4            bybit        SOLPERP         0.010       1.533085e+07
5            bybit        SOLUSDT         0.010       1.573076e+09
6   bitget_futures  SOLPERP_CMCBL         0.010       0.000000e+00
7   bitget_futures   SOLUSD_DMCBL         0.010       2.096455e+08
8   bitget_futures  SOLUSDT_UMCBL         0.012       1.367450e+09
9    coinw_futures       SOL/USDT         0.010       6.679367e+08


In [3]:
import pandas as pd

csv_path = r"C:\Users\FEMI\Desktop\Funding & Perps Dashboard\Exchange_database\perps_funding_comparison.csv"
df = pd.read_csv(csv_path)

print(df.head())

       exchange_id      exchange_name          symbol        base target  \
0  binance_futures  Binance (Futures)  1000000BOBUSDT  1000000BOB   USDT   
1  binance_futures  Binance (Futures)  1000000MOGUSDT  1000000MOG   USDT   
2  binance_futures  Binance (Futures)    1000BONKUSDC    1000BONK   USDC   
3  binance_futures  Binance (Futures)    1000BONKUSDT    1000BONK   USDT   
4  binance_futures  Binance (Futures)     1000CATUSDT     1000CAT   USDT   

   funding_rate  open_interest_usd    h24_volume   last  \
0         0.005       1.532888e+06  4.378201e+07  0.063   
1         0.005       5.293542e+06  1.094229e+07  1.004   
2         0.005       1.780992e+06  8.684559e+08  0.021   
3         0.005       5.749884e+07  7.401838e+09  0.021   
4        -0.029       4.135183e+06  1.757064e+09  0.008   

                                           trade_url  
0  https://www.binance.com/en/futures/1000000BOBUSDT  
1  https://www.binance.com/en/futures/1000000MOGUSDT  
2    https://www.binanc